<a href="https://colab.research.google.com/github/niz11/Bachelor_Thesis_Colab_Files/blob/main/3_frames_reapetVector_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Load encoding
X = np.load('drive/My Drive/dataset_3_frame_seq_faces/X_3.npy')
y = np.load('drive/My Drive/dataset_3_frame_seq_faces/Y_label_3.npy')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)


In [ ]:
import keras
from keras.layers import Input, Flatten, Dense,LSTM,TimeDistributed,RepeatVector
from keras.models import Model,Sequential
from keras.utils import plot_model


def create_model(encoding_length=64,sequence_length=10):
  visible = Input(shape=(sequence_length,encoding_length))
  encoder_outputs, state_h, state_c = LSTM(encoding_length, activation='relu', name='encoder', return_state=True)(visible)
  encoder_state = [state_h, state_c]
  clssifier  = Dense(7, activation='softmax')(encoder_outputs)

  model = Model(inputs=visible, outputs=[clssifier])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  plot_model(model, show_shapes=True, to_file='lstm_autoencoder.png')
  model.summary()
  return model

model = create_model(128, 3)
model.load_weights("drive/My Drive/trained_models/composite_repeat_3_frame_seq/encoderDotsRepeatVector-3.h5",by_name=True)

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3, 128)]          0         
_________________________________________________________________
encoder (LSTM)               [(None, 128), (None, 128) 131584    
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 903       
Total params: 132,487
Trainable params: 132,487
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=10, verbose=1, validation_split=0.05)

Epoch 1/10
80/80 [==============================] - 1s 15ms/step - loss: 1.4927 - accuracy: 0.4588 - val_loss: 1.0418 - val_accuracy: 0.6667
Epoch 2/10
80/80 [==============================] - 1s 12ms/step - loss: 0.7824 - accuracy: 0.7545 - val_loss: 0.6524 - val_accuracy: 0.8370
Epoch 3/10
80/80 [==============================] - 1s 11ms/step - loss: 0.4128 - accuracy: 0.8806 - val_loss: 0.4795 - val_accuracy: 0.8889
Epoch 4/10
80/80 [==============================] - 1s 12ms/step - loss: 0.1912 - accuracy: 0.9493 - val_loss: 0.4634 - val_accuracy: 0.9111
Epoch 5/10
80/80 [==============================] - 1s 12ms/step - loss: 0.0888 - accuracy: 0.9804 - val_loss: 0.5181 - val_accuracy: 0.9111
Epoch 6/10
80/80 [==============================] - 1s 12ms/step - loss: 0.0447 - accuracy: 0.9921 - val_loss: 0.5518 - val_accuracy: 0.9185
Epoch 7/10
80/80 [==============================] - 1s 11ms/step - loss: 0.0211 - accuracy: 0.9976 - val_loss: 0.5715 - val_accuracy: 0.9185
Epoch 8/10
80

In [ ]:
def getFaceExpressionFromIndex(i):
  if (i == 0):
      return 'surprise'
  elif (i == 1):
      return 'smile'
  elif (i == 2):
      return 'sad'
  elif (i == 3):
      return 'anger'
  elif (i == 4):
      return 'fear'
  elif (i == 5):
      return 'disgust'
  elif (i == 6):
      return 'none'
  else:
      print(i)

In [ ]:
test_predictions = model.predict(X_test)
correct = 0
for i in range(len(test_predictions)):
  truth = np.argmax(y_test[i])
  prediction = np.argmax(test_predictions[i])
  if truth == prediction:
    correct += 1
  else:
    print(f'Wrong classification, truth: {getFaceExpressionFromIndex(truth)}')
    print(f'Wrong classification, prediction: {getFaceExpressionFromIndex(prediction)}')
    print("---------------------------------------------------------------------------")
print(f'number of samples: {len(test_predictions)}')
print(f'correct: {correct}')
print(f'Accuracy of predicitons: {correct / len(test_predictions)}')

Wrong classification, truth: none
Wrong classification, prediction: fear
---------------------------------------------------------------------------
Wrong classification, truth: none
Wrong classification, prediction: sad
---------------------------------------------------------------------------
Wrong classification, truth: sad
Wrong classification, prediction: fear
---------------------------------------------------------------------------
Wrong classification, truth: fear
Wrong classification, prediction: none
---------------------------------------------------------------------------
Wrong classification, truth: fear
Wrong classification, prediction: sad
---------------------------------------------------------------------------
Wrong classification, truth: smile
Wrong classification, prediction: none
---------------------------------------------------------------------------
Wrong classification, truth: anger
Wrong classification, prediction: sad
-----------------------------------

In [ ]:
# Below start classifing the encodings

In [ ]:
from keras.models import load_model
from keras.models import model_from_json
import json

# load json and create model
json_file = open('drive/My Drive/trained_models/composite_repeat_3_frame_seq/encoderDotsRepeatVector_3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
encoder = model_from_json(loaded_model_json)
# load weights into new model
encoder.load_weights("drive/My Drive/trained_models/composite_repeat_3_frame_seq/encoderDotsRepeatVector-3.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
# Check here: better divinding to test/train set
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Load encoding
X = np.load('drive/My Drive/dataset_3_frame_seq_faces/X_3.npy')
y = np.load('drive/My Drive/dataset_3_frame_seq_faces/Y_label_3.npy')
predictions = encoder.predict(X)


X_train, X_test, y_train, y_test = train_test_split(predictions, y, test_size = 0.3, random_state = 0)
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD


def classifier():
  model = Sequential()
  model.add(Dense(64, input_dim=128, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(7, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model = classifier()
model.fit(X_train, y_train, epochs=50, verbose=1, validation_split=0.1)

Epoch 1/50
76/76 [==============================] - 0s 6ms/step - loss: 1.8028 - accuracy: 0.2471 - val_loss: 1.7625 - val_accuracy: 0.2639
Epoch 2/50
76/76 [==============================] - 0s 3ms/step - loss: 1.5899 - accuracy: 0.3964 - val_loss: 1.5584 - val_accuracy: 0.4164
Epoch 3/50
76/76 [==============================] - 0s 3ms/step - loss: 1.3743 - accuracy: 0.5124 - val_loss: 1.3385 - val_accuracy: 0.5316
Epoch 4/50
76/76 [==============================] - 0s 4ms/step - loss: 1.1563 - accuracy: 0.6045 - val_loss: 1.1867 - val_accuracy: 0.6245
Epoch 5/50
76/76 [==============================] - 0s 4ms/step - loss: 0.9872 - accuracy: 0.6828 - val_loss: 1.0624 - val_accuracy: 0.6617
Epoch 6/50
76/76 [==============================] - 0s 3ms/step - loss: 0.8503 - accuracy: 0.7425 - val_loss: 0.9641 - val_accuracy: 0.6840
Epoch 7/50
76/76 [==============================] - 0s 4ms/step - loss: 0.7303 - accuracy: 0.7765 - val_loss: 0.8957 - val_accuracy: 0.7286
Epoch 8/50
76/76 [==

In [ ]:
def getFaceExpressionFromIndex(i):
  if (i == 0):
      return 'surprise'
  elif (i == 1):
      return 'smile'
  elif (i == 2):
      return 'sad'
  elif (i == 3):
      return 'anger'
  elif (i == 4):
      return 'fear'
  elif (i == 5):
      return 'disgust'
  elif (i == 6):
      return 'none'
  else:
      print(i)

In [ ]:
test_predictions = model.predict(X_test)
correct = 0
for i in range(len(test_predictions)):
  truth = np.argmax(y_test[i])
  prediction = np.argmax(test_predictions[i])
  if truth == prediction:
    correct += 1
  else:
    print(f'Wrong classification, truth: {getFaceExpressionFromIndex(truth)}')
    print(f'Wrong classification, prediction: {getFaceExpressionFromIndex(prediction)}')
    print("---------------------------------------------------------------------------")

print(f'Accuracy of predicitons: {correct / len(test_predictions)}')
print(f'Got correct: {correct}')
print(f'Got wrong: {len(test_predictions) - correct}')

Wrong classification, truth: none
Wrong classification, prediction: fear
---------------------------------------------------------------------------
Wrong classification, truth: disgust
Wrong classification, prediction: anger
---------------------------------------------------------------------------
Wrong classification, truth: smile
Wrong classification, prediction: none
---------------------------------------------------------------------------
Wrong classification, truth: sad
Wrong classification, prediction: smile
---------------------------------------------------------------------------
Wrong classification, truth: none
Wrong classification, prediction: fear
---------------------------------------------------------------------------
Wrong classification, truth: fear
Wrong classification, prediction: none
---------------------------------------------------------------------------
Wrong classification, truth: fear
Wrong classification, prediction: sad
-----------------------------